# Decision Tree Classifier with Employee Attrition Dataset

In this notebook, we will build a decision tree classifier using the scikit-learn library. We will use a hypothetical employee attrition dataset for this example.

## Import Libraries
First, let's import the necessary libraries.

In [24]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score


## Load and Explore the Dataset
Next, we will load the employee attrition dataset and explore its contents.

In [2]:
# Load the dataset
df = pd.read_csv('employee_attrition.csv')

# Display the first few rows of the DataFrame
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,EducationField,Gender,HourlyRate,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,TotalWorkingYears
0,41,Yes,Travel_Rarely,1102,Sales,Life Sciences,Female,94,Sales Executive,4,Single,5993,19479,8,Yes,8
1,49,No,Travel_Frequently,279,Research & Development,Life Sciences,Male,61,Research Scientist,2,Married,5130,24907,1,No,10
2,37,Yes,Travel_Rarely,1373,Research & Development,Other,Male,92,Laboratory Technician,3,Single,2090,2396,6,Yes,7
3,33,No,Travel_Frequently,1392,Research & Development,Life Sciences,Female,56,Research Scientist,3,Married,2909,23159,1,Yes,8
4,27,No,Travel_Rarely,591,Research & Development,Medical,Male,40,Laboratory Technician,2,Married,3468,16632,9,No,6


## Preprocess the Data
We need to preprocess the data, including handling categorical variables and missing values.

In [3]:
# For simplicity, let's assume there are no missing values in this example.
# If there are, you can handle them with techniques like imputation or dropping rows/columns.

from sklearn.calibration import LabelEncoder

#Perform conversion of categorical variables
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the columns that does not have values
columns_to_convert = ['BusinessTravel','Department','EducationField','Gender','JobRole','JobSatisfaction','MaritalStatus','OverTime']
for column in columns_to_convert:
    df[column] = label_encoder.fit_transform(df[column])
df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,EducationField,Gender,HourlyRate,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,TotalWorkingYears
0,41,Yes,2,1102,2,1,0,94,7,3,2,5993,19479,8,1,8
1,49,No,1,279,1,1,1,61,6,1,1,5130,24907,1,0,10
2,37,Yes,2,1373,1,4,1,92,2,2,2,2090,2396,6,1,7
3,33,No,1,1392,1,1,0,56,6,2,1,2909,23159,1,1,8
4,27,No,2,591,1,3,1,40,2,1,1,3468,16632,9,0,6


## Split the Dataset
We will split the dataset into training and testing sets.

In [4]:
# Split the dataset into training and testing sets
y = df['Attrition']
# drop the outcome column
x = df.drop('Attrition', axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


##  Modify the code below to train the Decision Tree Classifier and leverage MLFlow to track 
### &emsp; 1. the experiment run 
### &emsp; 2. by logging hyperparameters, metrics.
### &emsp; 3. Register the model

In [21]:
#Import all the required mlflow related libraries

# modify the code below

mlflow.set_tracking_uri('http://127.0.0.1:5000')

with mlflow.start_run():
    # Create and train the decision tree classifier
    dt_classifier = DecisionTreeClassifier(max_depth=4,random_state=42)
    dt_classifier.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = dt_classifier.predict(X_test)
    
    # Calculate the accuracy of the decision tree model
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)

    mlflow.sklearn.log_model(
        sk_model=dt_classifier,
        name="model",
        registered_model_name="decision_tree_classifier"  # This triggers model registration
    )

    print(f"Logged to MLFlow: accuracy={acc:.4f}, f1={f1:.4f}")

2025/12/02 11:53:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'decision_tree_classifier' already exists. Creating a new version of this model...
2025/12/02 11:53:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: decision_tree_classifier, version 4
Created version '4' of model 'decision_tree_classifier'.


Logged to MLFlow: accuracy=0.8639, f1=0.5922
🏃 View run bald-skink-541 at: http://127.0.0.1:5000/#/experiments/0/runs/54833edb99964088b952f671788b2999
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


##  Load the Registered model and version
###  &emsp;Try to make a prediction with this loaded model

In [31]:
## Write your code here

model = mlflow.sklearn.load_model("models:/decision_tree_classifier/1")

sample = X_test.iloc[0].to_numpy().reshape(1, -1)
pred = model.predict(sample)
print("Predicted:", pred[0], " Actual:", y_test.iloc[0])

Predicted: No  Actual: No


/home/bounces/work/learn/nusiss-notes/IDAS/Courseware/workshop/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
